# Exploring Infogreffe (Financial Statements) Data

In [1]:
import pandas as pd
import psycopg2
import numpy as np
import configparser
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

### Initiate a connection to Redshift
using a local config file

In [2]:
config = configparser.ConfigParser()
project_config_path = '/Users/paulogier/Library/Preferences/PyCharm2019.3/scratches/capstone.cfg'
config.read(project_config_path)
DWH_HOST = config.get("DB", "DB_HOST")
DWH_DB = config.get("DB", "DB_NAME")
DWH_DB_USER = config.get("DB", "DB_USER")
DWH_DB_PASSWORD = config.get("DB", "DB_PASSWORD")
DWH_PORT = config.get("DB", "DB_PORT")
conn = psycopg2.connect(host =DWH_HOST ,database = DWH_DB, user=DWH_DB_USER, password=DWH_DB_PASSWORD, port=DWH_PORT)
conn.autocommit = True
print(conn.status)

1


### Loading the data

In [3]:
df = pd.read_sql("SELECT * FROM datalake.infogreffe_attributes;", conn)

In [4]:
df.sample(5)

,infogreffe_uid,siren,millesime,date_de_cloture,duree,tranche_ca_millesime,ca,resultat,effectif
1273099,71c34a3d7800c97fd66df1b86c3c1936,833193683,2018,2018-12-31,14.0,A - de 32K,0.0,-1567.0,NaN
1407649,ff8ee99d66b654592e7d9d85e0d6cf21,500556782,2018,2018-12-31,12.0,A - de 32K,NaN,NaN,NaN
1469195,e635cb31bc6d17cdb2e714fa2a55663e,493279046,2018,2018-09-30,12.0,A - de 32K,NaN,NaN,NaN
1130645,ef350d6cd4dbef1e04efcde30206f749,793286279,2018,2018-07-31,12.0,A - de 32K,NaN,NaN,NaN
1664711,eb9647e602d234053f3a4ddc2503a925,794533646,2018,2018-12-31,12.0,A - de 32K,NaN,NaN,NaN


### Number of rows

In [5]:
df.shape[0]

2395536

### Nulls

In [6]:
(df.isnull().sum(axis=0)/df.shape[0]).sort_values(ascending=False)

effectif                8.890261e-01
ca                      6.739485e-01
resultat                6.623144e-01
tranche_ca_millesime    1.252329e-06
duree                   8.348862e-07
date_de_cloture         8.348862e-07
infogreffe_uid          8.348862e-07
millesime               0.000000e+00
siren                   0.000000e+00
dtype: float64

Detailed information (revenue `ca`, profit `resultat`, number of employees `effectif`; is not available)

### Uniqueness of infogreffe_uid

In [7]:
if df['infogreffe_uid'].nunique()/df.shape[0] ==1:
    print('infogreffe_uid has unique values')

### Column statistics

In [8]:
df['millesime'].value_counts()

2019    1153295
2018    1011389
2020     230852
Name: millesime, dtype: int64

Financials available for 2018, 2019 and 2020 only (due to filters selected in the API)

In [9]:
df['tranche_ca_millesime'].value_counts()

A - de 32K             1464475
E + d 1M                499054
D entre 250K et 1M      184895
C entre 82K et 250K     162930
B entre 32K et 82K       84179
Name: tranche_ca_millesime, dtype: int64

Most of the firms have a revenue less than 32 k euros

In [10]:
x = df.pivot_table(index='tranche_ca_millesime', values='ca', aggfunc=lambda r: np.mean(r.isnull()))
x.rename(columns={'ca': 'ca_pct_null'}, inplace=True)
x

,ca_pct_null
tranche_ca_millesime,
A - de 32K,0.932511
B entre 32K et 82K,0.000000
C entre 82K et 250K,0.000000
D entre 250K et 1M,0.000000
E + d 1M,0.498595


Curiously, even for big firms (revenue > 1m), the revenue is not always indicated...    
I would have to understand why